In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix
# Qiskit Imports
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_algorithms.optimizers import COBYLA
from qiskit_machine_learning.algorithms import VQC
from qiskit.primitives import Sampler

# --- 1. PREPROCESAMIENTO ---
# Cargar datos
df = pd.read_csv('/content/sample_data/dataset_proyecto.csv')
df = df.drop(columns=['ID', 'Dt_Customer']).dropna()

# Definir variables
target = 'Response'
X = pd.get_dummies(df.drop(columns=[target]), drop_first=True)
y = df[target].values

# Escalar y PCA (4 Qubits)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components=4)
X_pca = pca.fit_transform(X_scaled)

# Muestreo "Ligero" para Simulación (200 datos)
X_small, y_small = resample(X_pca, y, n_samples=200, random_state=42, stratify=y)
X_train, X_test, y_train, y_test = train_test_split(X_small, y_small, test_size=0.3, random_state=42)

# --- 2. MODELO CUÁNTICO (VQC) ---
num_features = X_train.shape[1]
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)
ansatz = RealAmplitudes(num_qubits=num_features, reps=1)
optimizer = COBYLA(maxiter=60)

vqc = VQC(feature_map=feature_map, ansatz=ansatz, optimizer=optimizer, sampler=Sampler())

print("Entrenando VQC...")
vqc.fit(X_train, y_train)
score_vqc = vqc.score(X_test, y_test)
print(f"Precisión VQC: {score_vqc:.2f}")

# --- 3. MODELO CLÁSICO (Random Forest) ---
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
score_rf = rf.score(X_test, y_test)
print(f"Precisión Random Forest: {score_rf:.2f}")

/tmp/ipython-input-704032133.py:42: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  vqc = VQC(feature_map=feature_map, ansatz=ansatz, optimizer=optimizer, sampler=Sampler())


Entrenando VQC...
Precisión VQC: 0.50
Precisión Random Forest: 0.90


In [ ]:
!pip install qiskit qiskit-machine-learning

El error `ModuleNotFoundError: No module named 'qiskit'` ocurre porque la librería Qiskit no está instalada en tu entorno de Colab. Las celdas anteriores instalan `qiskit` y `qiskit-machine-learning`, que son necesarias para ejecutar el código que utiliza Variational Quantum Classifiers (VQC).